## NOTEBOOK IMPORTS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings

## CONSTANTS

In [78]:
VOCAB_SIZE=2500
TRAINING_DATA_FILE='SpamData/02_Training/train-data.txt'
TEST_DATA_FILE='SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE='SpamData/03_Testing/prob_spam.txt'
TOKEN_HAM_PROB_FILE='SpamData/03_Testing/prob_nonspam.txt'
TOKEN_ALL_PROB_FILE='SpamData/03_Testing/prob_all.txt'

TEST_FEATURE_MATRIX='SpamData/03_Testing/test_features.txt'
TEST_TARGET_FILE='SpamData/03_Testing/test_target.txt'

## READ AND LOAD FEATURES FROM .txt FILES INTO NUMPY ARRAY

In [3]:
sparse_train_data=np.loadtxt(TRAINING_DATA_FILE,delimiter=' ',dtype=int)
sparse_test_data=np.loadtxt(TEST_DATA_FILE,delimiter=' ',dtype=int)

In [4]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [5]:
print('No. of rows in train data',sparse_train_data.shape[0])
print('No. of rows in test data',sparse_test_data.shape[0])

No. of rows in train data 258353
No. of rows in test data 117649


In [6]:
print('Number of emails in train set', np.unique(sparse_train_data[:,0]).size)
print('Number of emails in test set', np.unique(sparse_test_data[:,0]).size)

Number of emails in train set 4015
Number of emails in test set 1724


## CREATING EMPTY DATAFRAME

In [7]:
column_names=['DOC_ID','CATEGORY']+list(range(VOCAB_SIZE))

In [8]:
len(column_names)

2502

In [9]:
index_names=np.unique(sparse_train_data[:,0])

In [10]:
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [11]:
fully_trained_data=pd.DataFrame(columns=column_names,index=index_names)

In [12]:
fully_trained_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
fully_trained_data.fillna(0,inplace=True)

In [14]:
fully_trained_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## CREATING A FULL MATRIX FROM SPARSE MATRIX

In [15]:
def  make_full_matrix(sparsed_matix,nr_words,doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    column_names=['DOC_ID','CATEGORY']+list(range(nr_words))
    index_names=np.unique(sparsed_matix[:,0])
    full_matrix=pd.DataFrame(columns=column_names,index=index_names)
    full_matrix.fillna(0,inplace=True)
    
    for i in range(sparsed_matix.shape[0]):
        doc_nr=sparsed_matix[i,doc_idx]
        word_id=sparsed_matix[i,word_idx]
        label=sparsed_matix[i,cat_idx]
        occurence=sparsed_matix[i,freq_idx]
        
        full_matrix['DOC_ID'].at[doc_nr]=doc_nr
        full_matrix['CATEGORY'].at[doc_nr]=label
        full_matrix[word_id].at[doc_nr]=occurence
    full_matrix.set_index('DOC_ID',inplace=True)
    return full_matrix

In [16]:
full_train_data=make_full_matrix(sparse_train_data,VOCAB_SIZE)

In [17]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,13,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## TRAINING THE NAIVE BAYES MODEL

## CALCULATING THE PROBABILITY OF SPAM

In [18]:
prob_spam=full_train_data[full_train_data['CATEGORY']==1].shape[0]/full_train_data.shape[0]

In [19]:
prob_ham=full_train_data[full_train_data['CATEGORY']==0].shape[0]/full_train_data.shape[0]

In [20]:
print(prob_spam,prob_ham)

0.31133250311332505 0.688667496886675


## TOTAL NUMBER OF WORDS / TOKENS

In [21]:
full_train_features=full_train_data.iloc[:,full_train_data.columns!='CATEGORY']

In [22]:
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
email_len=full_train_features.sum(axis=1)

In [24]:
email_len.shape

(4015,)

In [25]:
email_len[:5]

DOC_ID
0     87
1     53
2     40
3    182
4     43
dtype: int64

In [26]:
total_wc=email_len.sum()

In [27]:
total_wc

430966

## NUMBER OF TOKENS IN SPAM AND HAM EMAILS

In [28]:
spam_emails=full_train_data[full_train_data['CATEGORY']==1]

In [29]:
spam_emails=spam_emails.iloc[:,spam_emails.columns!='CATEGORY']

In [30]:
spam_emails

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
ham_emails=full_train_data[full_train_data['CATEGORY']==0]

In [32]:
ham_emails=ham_emails.iloc[:,ham_emails.columns!='CATEGORY']

In [33]:
ham_emails

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
spam_lenghts=spam_emails.sum(axis=1)

In [35]:
spam_wc=spam_lenghts.sum()

In [37]:
spam_wc

178573

In [38]:
ham_lengths=ham_emails.sum(axis=1)
nonspam_wc=ham_lengths.sum()
nonspam_wc

252393

## SUMMING THE TOKENS OCCURING IN SPAM

In [39]:
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
train_spam_tokens=spam_emails

In [42]:
train_spam_tokens.shape

(1250, 2500)

In [50]:
summed_spam_tokens=train_spam_tokens.sum(axis=0)+1

In [51]:
summed_spam_tokens

0       2177
1        936
2       1220
3       2026
4       1211
        ... 
2495       5
2496       3
2497       5
2498       2
2499      28
Length: 2500, dtype: int64

## SUMMING THE TOKENS OCCURING IN HAM

In [52]:
train_ham_tokens=ham_emails

In [53]:
train_ham_tokens.shape

(2765, 2500)

In [54]:
summed_ham_tokens=train_ham_tokens.sum(axis=0)+1

In [55]:
summed_ham_tokens

0       5483
1       2588
2       2044
3        938
4       1611
        ... 
2495      37
2496      27
2497      30
2498      37
2499       5
Length: 2500, dtype: int64

## P(TOKEN/SPAM)  -  PROBABILTY THAT TOKEN OCCURS GIVEN THE EMAIL IS SPAM

In [63]:
prob_tokens_spam=summed_spam_tokens/(spam_wc+VOCAB_SIZE)

In [65]:
prob_tokens_spam[:5]

0    0.012023
1    0.005169
2    0.006738
3    0.011189
4    0.006688
dtype: float64

In [64]:
prob_tokens_spam.sum()

1.0

## P(TOKEN/HAM) - PROBABILITY THAT TOKEN OCCURS GIVEN EMAIL IS HAM

In [66]:
prob_tokens_ham=summed_ham_tokens/(nonspam_wc+VOCAB_SIZE)

In [69]:
prob_tokens_ham[-5:]

2495    0.000145
2496    0.000106
2497    0.000118
2498    0.000145
2499    0.000020
dtype: float64

In [70]:
prob_tokens_ham.sum()

1.0

## P(TOKEN) - PROBABILITY THAT TOKEN OCCURS

In [72]:
prob_tokens_all=full_train_features.sum(axis=0)/total_wc

In [73]:
prob_tokens_all[:5]

0    0.017769
1    0.008172
2    0.007569
3    0.006873
4    0.006543
dtype: float64

In [74]:
prob_tokens_all.sum()

0.9999999999999999

## SAVE TRAINING MODEL

In [77]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_tokens_all)

## PREPARE TEST DATA

In [82]:
full_test_data=make_full_matrix(sparse_test_data,nr_words=VOCAB_SIZE)

In [83]:
full_test_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
X_test=full_test_data.loc[:,full_test_data.columns!='CATEGORY']

In [86]:
y_test=full_test_data.CATEGORY

In [87]:
np.savetxt(TEST_TARGET_FILE,y_test)
np.savetxt(TEST_FEATURE_MATRIX,X_test)